# BiqQuery to Cloud Storage

This notebook shows how to execute a query against bigquery and store the results in BigQuery.

In [ ]:
# Libraries ... 
from google.cloud import bigquery
import datetime
import utils.bq_to_storage

In [ ]:
# Query Parameters
FILE = 'ethereum-transacting-partner-count'
SQL_PATH = 'sql/' + FILE + '.sql'

# Storage Parameters
BUCKET_URI = 'gs://blockchain-exploration/queries'
todaysDate = datetime.date.today().strftime('%Y-%m-%d');

In [ ]:
bqclient = bigquery.Client()

In [ ]:
QUERY = open(SQL_PATH).read()

In [ ]:
dataframe = (
    bqclient.query(QUERY)
    .result()
    .to_dataframe(
        create_bqstorage_client=True,
    )
)

In [ ]:
filename = "{}/{}-{}.csv".format(BUCKET_URI, todaysDate, FILE)
dataframe.to_csv(filename)

In [ ]:
# Test function with new params
bq_to_storage('bitcoin-zero-fee')

In [ ]:
%%bigquery df
WITH double_entry_book AS (
   -- debits
   SELECT
    array_to_string(inputs.addresses, ",") as address
   , inputs.type
   , -inputs.value as value
   FROM `bigquery-public-data.crypto_bitcoin.inputs` as inputs
   UNION ALL
   -- credits
   SELECT
    array_to_string(outputs.addresses, ",") as address
   , outputs.type
   , outputs.value as value
   FROM `bigquery-public-data.crypto_bitcoin.outputs` as outputs
)
SELECT
   address
,   type   
,   sum(value) as balance
FROM double_entry_book
GROUP BY 1,2
ORDER BY balance DESC
LIMIT 1000